<a href="https://colab.research.google.com/github/mj3876/-/blob/main/DSP2025_AFTER_TTA_VERSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [디지탈 신호 처리] (2025)


---


### 딥러닝 프로젝트 : 딥러닝 기반 화자 식별 알고리즘 성능 향상

---
*   다양한 방법을 활용하여 모델 성능을 높이면 됩니다. (DSP 기법 활용, 데이터 전처리 및 증강, 모델 구조 개선, 학습 기법 개선, 손실 함수 개선)
* 데이터셋 : 1초 단위의 음성이 약 4,648개로, Train : Validation : Test = 8 : 1 : 1로 구분되어 있습니다. 여러분들은 테스트 세트에서 가장 좋은 성능 (Accuracy)을 보이는 것이 목표입니다.
* 환경 구성 : 런어스 혹은 깃허브 (https://github.com/JaeBinCHA7/DNN-based-Speaker-Identification-Tutorial) 에서 코드와 데이터셋을 다운로드 합니다. 다운로드한 데이터셋과 코드는 구글 드라이브에 올려주세요.
* GPU가 탑재된 로컬 PC가 없는 분들은.ipynb 를 활용해주시고, 가용할 수 있는 GPU가 있는 분들은 깃허브 코드를 참고해주시면 됩니다.
* 제출 : 여러분들이 학습 모델을 테스트셋에서 추론한 후 얻은 "test_predictions.csv"를 런어스 자료실에 업로드 해주세요.





---


### 구글 드라이브 마운트



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change Directory
%cd '/content/drive/My Drive/dsp/project'

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/dsp_dataset/dsp_dataset_2025 (3).zip'
extract_path = '/content/drive/MyDrive/dsp_dataset'
dataset_dir = os.path.join(extract_path, 'dataset')

if not os.path.exists(dataset_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"'{zip_path}' extracted to '{extract_path}'")
else:
    print(f"'{dataset_dir}' already exists. Skipping extraction.")



---


### 필요한 라이브러리 설치하기


In [ ]:
pip install tensorboardX

In [ ]:
import argparse
import os
import time
import torch
import numpy as np
from tensorboardX import SummaryWriter
import torch.nn as nn
import torch.nn.functional as F
import os
import argparse
import torch
import time
import numpy as np
from torch.utils.data import DataLoader, Dataset
import random
import pandas as pd
from typing import Dict
import json
from tqdm.auto import tqdm
import librosa
import soundfile as sf
import math

In [ ]:


# Cell 2: 묵음 제거 함수 정의
import os
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
from tqdm.auto import tqdm


def remove_silence(
    audio_path: str,
    output_path: str,
    min_silence_len: int = 300,
    silence_thresh: int = -40,
    keep_silence: int = 100,
    min_audio_len: int = 500,
):
    """
    오디오에서 묵음 제거

    Parameters:
        min_silence_len: 묵음으로 간주할 최소 길이 (ms)
        silence_thresh: 묵음 threshold (dBFS, 낮을수록 엄격)
        keep_silence: 앞뒤로 남길 묵음 (ms)
        min_audio_len: 최소 오디오 길이 (ms)
    """
    try:
        audio = AudioSegment.from_file(audio_path)

        # 묵음 기준으로 분할
        chunks = split_on_silence(
            audio,
            min_silence_len=min_silence_len,
            silence_thresh=silence_thresh,
            keep_silence=keep_silence,
        )

        # 청크가 없으면 원본 유지
        if len(chunks) == 0:
            audio.export(output_path, format="wav")
            return len(audio), len(audio), 0

        # 모든 청크 이어붙이기
        combined = AudioSegment.empty()
        for chunk in chunks:
            combined += chunk

        # 너무 짧으면 원본 유지
        if len(combined) < min_audio_len:
            audio.export(output_path, format="wav")
            return len(audio), len(audio), 0

        # 저장
        combined.export(output_path, format="wav")

        return len(audio), len(combined), len(audio) - len(combined)

    except Exception as e:
        print(f"  ❌ Error: {e}")
        return 0, 0, 0

def process_dataset(
    csv_path: str,
    input_root: str,
    output_root: str,
    min_silence_len: int = 300,
    silence_thresh: int = -40,
):
    """CSV 기반 데이터셋 전체 처리"""

    df = pd.read_csv(csv_path)
    print(f"📁 Processing {len(df)} files from {csv_path}")

    os.makedirs(output_root, exist_ok=True)

    total_orig = 0
    total_new = 0
    success = 0

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Removing silence"):
        rel_path = row['path']
        input_path = os.path.join(input_root, rel_path)
        output_path = os.path.join(output_root, rel_path)

        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        orig_len, new_len, removed = remove_silence(
            input_path, output_path,
            min_silence_len=min_silence_len,
            silence_thresh=silence_thresh,
        )

        if new_len > 0:
            success += 1
            total_orig += orig_len
            total_new += new_len




def process_all_datasets(
    dataset_root: str = "./dataset",
    output_root: str = "./dataset_no_silence",
    min_silence_len: int = 300,
    silence_thresh: int = -40,
):
    """train, valid, test 모두 처리"""


    for split in ['train', 'valid', 'test']:
        csv_path = os.path.join(dataset_root, f"{split}.csv")

        if not os.path.exists(csv_path):
            print(f"⚠️  Skipping {split}.csv (not found)")
            continue

        print(f"\n{'#'*60}")
        print(f"Processing {split.upper()} dataset")
        print(f"{'#'*60}")

        process_dataset(
            csv_path=csv_path,
            input_root=dataset_root,
            output_root=output_root,
            min_silence_len=min_silence_len,
            silence_thresh=silence_thresh,
        )

    print(f"\n{'='*60}")
    print(f"All datasets processed!")
    print(f"{'='*60}")





# Cell 5: 전체 데이터셋 묵음 제거 실행
# 주의: 시간이 오래 걸릴 수 있습니다!
process_all_datasets(
    dataset_root="/content/drive/MyDrive/dsp_dataset/dataset",
    output_root="/content/drive/MyDrive/dsp_dataset/dataset/dataset_no_silence",
    min_silence_len=300,  # 필요시 조정
    silence_thresh=-40    # 필요시 조정
)






---


### 훈련 하이퍼 파라미터 설정



In [ ]:
def build_args():
    p = argparse.ArgumentParser(description="화자 식별 모델 학습")

    # 실험 설정
    p.add_argument("--exp_name", type=str, default="EXP_LS_TTA",
                   help="실험 이름")

    # 데이터 경로
    dataset_root_default = "/content/drive/MyDrive/dsp_dataset/dataset"
    p.add_argument("--dataset_root", type=str, default=dataset_root_default, help="데이터셋 루트 경로")
    p.add_argument("--train_csv", type=str, default=os.path.join(dataset_root_default, "train.csv"))
    p.add_argument("--valid_csv", type=str, default=os.path.join(dataset_root_default, "valid.csv"))
    p.add_argument("--test_csv", type=str, default=os.path.join(dataset_root_default, "test.csv"))



    # 오디오 / Feature
    p.add_argument("--sr", type=int, default=16000)
    p.add_argument("--duration", type=float, default=1.0, help="샘플 길이 (초)")
    p.add_argument("--n_mels", type=int, default=80)
    p.add_argument("--n_fft", type=int, default=400)
    p.add_argument("--hop_length", type=int, default=160)

    # 학습 설정
    p.add_argument("--epochs", type=int, default=40)
    p.add_argument("--batch_size", type=int, default=32)
    p.add_argument("--lr", type=float, default=1e-3)
    p.add_argument("--min_lr", type=float, default=1e-6)
    p.add_argument("--weight_decay", type=float, default=1e-4)
    p.add_argument("--grad_clip", type=float, default=5.0)
    p.add_argument("--num_workers", type=int, default=0)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--cpu", action="store_true")

    # 모델 구조
    p.add_argument("--encoder_dim", type=int, default=256)
    p.add_argument("--num_blocks", type=int, default=6)
    p.add_argument("--num_heads", type=int, default=4)
    p.add_argument("--ff_expansion", type=int, default=4)
    p.add_argument("--conv_kernel_size", type=int, default=15)
    p.add_argument("--dropout", type=float, default=0.1)

    # Label Smoothing
    p.add_argument("--label_smoothing", type=float, default=0.1,
                   help="Label smoothing 계수")

    # 출력 경로
    p.add_argument("--out_dir", type=str, default="./logs", help="로그 저장 경로")

    return p.parse_known_args()

---

### Helper functions

In [ ]:
# 유틸리티 함수
def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def load_csv(csv_path: str) -> pd.DataFrame:
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV not found: {csv_path}")
    df = pd.read_csv(csv_path)

    # Test 파일은 speaker_id 없을 수 있음
    if "test" not in csv_path.lower():
        required = {"path", "speaker_id"}
        if not required.issubset(df.columns):
            raise ValueError(f"{csv_path} must contain columns: {required}")
    else:
        if "path" not in df.columns:
            raise ValueError(f"{csv_path} must contain 'path' column")

    return df


def build_label_map(train_df: pd.DataFrame, valid_df: pd.DataFrame) -> Dict[str, int]:
    spk_all = sorted(set(train_df["speaker_id"].astype(str)) | set(valid_df["speaker_id"].astype(str)))
    label_map = {spk: i for i, spk in enumerate(spk_all)}
    return label_map


def get_lr(optimizer: torch.optim.Optimizer) -> float:
    for pg in optimizer.param_groups:
        return float(pg.get("lr", 0.0))
    return 0.0


def fmt_pct(x: float) -> str:
    return f"{100.0 * x:6.2f}%"


def print_epoch_header(epoch: int, total_epochs: int):
    bar = "=" * 66
    print(f"\n{bar}\n[Epoch {epoch:03d}/{total_epochs:03d}] START\n{bar}")


def print_epoch_summary(epoch: int, train_loss: float, train_acc: float,
                        val_loss: float, val_acc: float, best_acc: float,
                        lr: float, elapsed: float, ckpt_best: bool, csv_path: str):
    # Epoch 결과 출력
    bar = "-" * 66
    print(f"{bar}")
    print(f"| Split |   Loss   |   Acc    |")
    print(f"{bar}")
    print(f"| Train | {train_loss:8.4f} | {fmt_pct(train_acc)} |")
    print(f"| Valid | {val_loss:8.4f} | {fmt_pct(val_acc)} |")
    print(f"{bar}")
    print(f"| Best Acc: {fmt_pct(best_acc)} | LR: {lr:.3e} | Time: {elapsed:.1f}s |")
    print(f"{bar}")
    if ckpt_best:
        print("✓ New BEST model saved.")
    print("=" * 66)

    # CSV 로깅
    row = dict(
        epoch=epoch,
        train_loss=float(train_loss),
        train_acc=float(train_acc),
        valid_loss=float(val_loss),
        valid_acc=float(val_acc),
        best_acc=float(best_acc),
        lr=float(lr),
        time_sec=float(elapsed),
    )
    header = not os.path.exists(csv_path)
    df = pd.DataFrame([row])
    ensure_dir(os.path.dirname(csv_path))
    df.to_csv(csv_path, mode="a", index=False, header=header)


# 오디오 처리
def load_wav(full_path: str, target_sr: int) -> np.ndarray:
    try:
        y, sr = sf.read(full_path, always_2d=False)
        if y.ndim > 1:
            y = np.mean(y, axis=1)  # mixdown to mono
        if sr != target_sr:
            y = librosa.resample(y, orig_sr=sr, target_sr=target_sr, res_type="kaiser_best")
        return y.astype(np.float32)
    except Exception:
        y, _ = librosa.load(full_path, sr=target_sr, mono=True)
        return y.astype(np.float32)


def fix_length(y: np.ndarray, num_samples: int) -> np.ndarray:
    if len(y) == num_samples:
        return y
    if len(y) > num_samples:
        return y[:num_samples]
    pad = num_samples - len(y)
    return np.pad(y, (0, pad), mode="constant")


def wav_to_logmel(
        y: np.ndarray,
        sr: int,
        n_mels: int = 80,
        n_fft: int = 400,  # 25 ms @ 16k
        hop_length: int = 160,  # 10 ms @ 16k
        fmin: int = 20,
        fmax: int = 7600,
) -> np.ndarray:
    S = librosa.feature.melspectrogram(
        y=y, sr=sr, n_fft=n_fft, hop_length=hop_length,
        n_mels=n_mels, fmin=fmin, fmax=fmax, power=2.0
    )
    S_db = librosa.power_to_db(S, ref=np.max)
    return S_db.astype(np.float32)  # [n_mels, T]


# Cepstral mean and variance normalization
def cmvn(x: np.ndarray, eps: float = 1e-5) -> np.ndarray:
    mu = x.mean(axis=1, keepdims=True)
    std = x.std(axis=1, keepdims=True)
    return (x - mu) / (std + eps)


# ============================================
# [Label Smoothing Cross Entropy]
# ============================================
class LabelSmoothingCrossEntropy(nn.Module):
    """
    Label Smoothing Cross Entropy Loss

    일반 Cross Entropy는 정답 클래스에 100% 확률을 부여하는데,
    Label Smoothing은 정답에 (1-ε)의 확률을, 나머지 클래스에 ε/(K-1)의 확률을 부여합니다.

    효과:
    - Overfitting 방지
    - 모델이 과도하게 확신하는 것을 방지 (calibration 개선)
    - 일반화 성능 향상

    Args:
        smoothing: smoothing 정도 (0.0 ~ 1.0)
                   0.0이면 일반 CrossEntropy와 동일
                   보통 0.1 정도 사용
    """
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.confidence = 1.0 - smoothing

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (pred.size(-1) - 1))
            true_dist.scatter_(1, target.unsqueeze(1), self.confidence)

        return torch.mean(torch.sum(-true_dist * pred, dim=-1))


---

### 데이터로더



In [ ]:
import soundfile as sf
import torchaudio

def load_audio_safe(path: str, target_sr: int = 16000):
    """soundfile을 직접 사용하여 오디오 로드"""
    try:
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")

        waveform, sr = sf.read(path, dtype='float32')

        # mono로 변환
        if len(waveform.shape) > 1:
            waveform = waveform.mean(axis=1)

        waveform = torch.from_numpy(waveform).unsqueeze(0)  # [1, T]

        # 리샘플링
        if sr != target_sr:
            waveform = torchaudio.functional.resample(waveform, sr, target_sr)

        return waveform, target_sr

    except Exception as e:
        print(f"Warning: Failed to load {path}: {e}")
        silent_waveform = torch.zeros(1, target_sr)
        return silent_waveform, target_sr


# For training
class SpeakerDataset(Dataset):
    def __init__(
            self,
            csv_path: str,
            dataset_root: str = "./dataset",
            label_map: Dict[str, int] = None,
            target_sr: int = 16000,
            duration_sec: float = 1.0,
            n_mels: int = 80,
            n_fft: int = 400,
            hop_length: int = 160,
            apply_cmvn: bool = True,
    ):
        self.df = load_csv(csv_path).copy()
        self.df["abs_path"] = self.df["path"].apply(lambda p: os.path.join(dataset_root, p))
        self.label_map = label_map if label_map else build_label_map(self.df)
        self.df["label"] = self.df["speaker_id"].astype(str).map(self.label_map)

        before = len(self.df)
        self.df = self.df.dropna(subset=["label"])
        self.df["label"] = self.df["label"].astype(int)
        after = len(self.df)
        if after < before:
            print(f"[WARN] Dropped {before - after} rows due to unmapped speaker_id.")

        self.target_sr = target_sr
        self.target_len = int(duration_sec * target_sr)
        self.mel_spec = torchaudio.transforms.MelSpectrogram(
            sample_rate=target_sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels
        )
        self.apply_cmvn = apply_cmvn

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        waveform, sr = load_audio_safe(row["abs_path"], self.target_sr)

        wav_len = waveform.size(1)
        if wav_len < self.target_len:
            waveform = F.pad(waveform, (0, self.target_len - wav_len))
        elif wav_len > self.target_len:
            # Random crop (핵심!)
            start = random.randint(0, wav_len - self.target_len)
            waveform = waveform[:, start:start + self.target_len]

        mel = self.mel_spec(waveform)
        mel = torch.log(mel + 1e-9)
        if self.apply_cmvn:
            mel = (mel - mel.mean()) / (mel.std() + 1e-9)

        return mel, row["label"]


# For test
class TestDataset(Dataset):
    def __init__(
        self,
        csv_path: str,
        dataset_root: str = "./dataset",
        target_sr: int = 16000,
        duration_sec: float = 1.0,
        n_mels: int = 80,
        n_fft: int = 400,
        hop_length: int = 160,
        apply_cmvn: bool = True,
    ):
        self.df = load_csv(csv_path).copy()
        self.df["abs_path"] = self.df["path"].apply(lambda p: os.path.join(dataset_root, p))
        self.target_sr = target_sr
        self.target_len = int(duration_sec * target_sr)
        self.mel_spec = torchaudio.transforms.MelSpectrogram(
            sample_rate=target_sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels
        )
        self.apply_cmvn = apply_cmvn

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        waveform, sr = load_audio_safe(row["abs_path"], self.target_sr)

        wav_len = waveform.size(1)
        if wav_len < self.target_len:
            waveform = F.pad(waveform, (0, self.target_len - wav_len))
        else:
            waveform = waveform[:, :self.target_len]

        mel = self.mel_spec(waveform)
        mel = torch.log(mel + 1e-9)
        if self.apply_cmvn:
            mel = (mel - mel.mean()) / (mel.std() + 1e-9)

        return mel, os.path.basename(row["path"])


---

### 모델 선언

In [ ]:
# ============================================
# MFA-Conformer Components
# ============================================

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)


class ConformerBlock(nn.Module):
    """Conformer Block"""
    def __init__(self, dim: int, num_heads: int = 4, ff_expansion: int = 4,
                 conv_kernel_size: int = 15, dropout: float = 0.1):
        super().__init__()
        # First Feed Forward (Macaron-style)
        self.ffn1 = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, ff_expansion * dim),
            Swish(),
            nn.Dropout(dropout),
            nn.Linear(ff_expansion * dim, dim),
            nn.Dropout(dropout)
        )

        # Multi-Head Self-Attention
        self.ln_attn = nn.LayerNorm(dim)
        self.mha = nn.MultiheadAttention(dim, num_heads, dropout=dropout, batch_first=True)
        self.dropout_attn = nn.Dropout(dropout)

        # Convolution Module
        self.ln_conv = nn.LayerNorm(dim)
        self.pointwise_conv1 = nn.Conv1d(dim, 2 * dim, kernel_size=1)
        self.depthwise_conv = nn.Conv1d(dim, dim, kernel_size=conv_kernel_size,
                                       padding=(conv_kernel_size - 1) // 2, groups=dim)
        self.bn = nn.BatchNorm1d(dim)
        self.swish = Swish()
        self.pointwise_conv2 = nn.Conv1d(dim, dim, kernel_size=1)
        self.dropout_conv = nn.Dropout(dropout)

        # Second Feed Forward
        self.ffn2 = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, ff_expansion * dim),
            Swish(),
            nn.Dropout(dropout),
            nn.Linear(ff_expansion * dim, dim),
            nn.Dropout(dropout)
        )

        self.norm_out = nn.LayerNorm(dim)

    def forward(self, x):
        # x: [B, T, D]
        # Feed Forward 1 (half step)
        x = x + 0.5 * self.ffn1(x)

        # Multi-Head Self-Attention
        x_normed = self.ln_attn(x)
        attn_out, _ = self.mha(x_normed, x_normed, x_normed)
        x = x + self.dropout_attn(attn_out)

        # Convolution Module
        residual = x
        x = self.ln_conv(x)
        x = x.transpose(1, 2)  # [B, D, T]
        x = self.pointwise_conv1(x)
        x = F.glu(x, dim=1)  # [B, D, T]
        x = self.depthwise_conv(x)
        x = self.bn(x)
        x = self.swish(x)
        x = self.pointwise_conv2(x)
        x = self.dropout_conv(x)
        x = x.transpose(1, 2)  # [B, T, D]
        x = residual + x

        # Feed Forward 2 (half step)
        x = x + 0.5 * self.ffn2(x)

        return self.norm_out(x)


class AttentiveStatsPooling(nn.Module):
    """Attentive Statistics Pooling"""
    def __init__(self, dim: int):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(dim, dim),
            nn.Tanh(),
            nn.Linear(dim, 1)
        )

    def forward(self, x):
        # x: [B, T, D]
        attn_scores = self.attention(x)  # [B, T, 1]
        attn_weights = F.softmax(attn_scores, dim=1)  # [B, T, 1]

        # Weighted mean
        weighted_mean = torch.sum(x * attn_weights, dim=1)  # [B, D]

        # Weighted std
        mean = torch.mean(x, dim=1, keepdim=True)  # [B, 1, D]
        variance = torch.sum(attn_weights * (x - mean) ** 2, dim=1)  # [B, D]
        weighted_std = torch.sqrt(variance + 1e-9)

        # Concatenate mean and std
        pooled = torch.cat([weighted_mean, weighted_std], dim=-1)  # [B, 2*D]
        return pooled


class MFAConformerSpeakerNet(nn.Module):
    """Multi-scale Feature Aggregation Conformer"""
    def __init__(self, n_classes: int, input_dim: int = 80, encoder_dim: int = 256,
                 num_blocks: int = 6, num_heads: int = 4, ff_expansion: int = 4,
                 conv_kernel_size: int = 15, dropout: float = 0.1, embedding_dim: int = 192):
        super().__init__()

        # Convolution Subsampling
        self.conv_subsample = nn.Sequential(
            nn.Conv2d(1, encoder_dim, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(encoder_dim, encoder_dim, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )

        # Linear projection
        self.input_proj = nn.Linear((input_dim // 4) * encoder_dim, encoder_dim)

        # Conformer Blocks
        self.blocks = nn.ModuleList([
            ConformerBlock(encoder_dim, num_heads, ff_expansion, conv_kernel_size, dropout)
            for _ in range(num_blocks)
        ])

        # Multi-scale Feature Aggregation
        self.mfa_norm = nn.LayerNorm(encoder_dim * num_blocks)

        # Attentive Statistics Pooling
        self.asp = AttentiveStatsPooling(encoder_dim * num_blocks)

        # Embedding layer
        self.embedding = nn.Sequential(
            nn.Linear(encoder_dim * num_blocks * 2, embedding_dim),
            nn.BatchNorm1d(embedding_dim)
        )

        # Classifier
        self.classifier = nn.Linear(embedding_dim, n_classes)

    def forward(self, x):
        # x: [B, 1, F, T]
        x = self.conv_subsample(x)  # [B, C, F/4, T/4]

        B, C, F, T = x.shape
        x = x.permute(0, 3, 1, 2).contiguous().view(B, T, C * F)  # [B, T, C*F]
        x = self.input_proj(x)  # [B, T, D]

        # Multi-scale Feature Aggregation
        block_outputs = []
        for block in self.blocks:
            x = block(x)
            block_outputs.append(x)

        # Concatenate all block outputs
        x = torch.cat(block_outputs, dim=-1)  # [B, T, D*num_blocks]
        x = self.mfa_norm(x)

        # Attentive Statistics Pooling
        x = self.asp(x)  # [B, 2*D*num_blocks]

        # Embedding
        emb = self.embedding(x)  # [B, embedding_dim]

        # Classification
        logits = self.classifier(emb)

        return logits

---

### 학습 트레이너

In [ ]:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.backends.cudnn.benchmark = True

# args를 수정해서 새 데이터셋 경로 사용
args = build_args()
args[0].exp_name = "EXP_NoSilence"
args[0].dataset_root = "/content/dataset_no_silence/"  # 변경된 경로
args[0].epochs = 50

class Trainer:
    def __init__(self, args):
        self.args = args
        set_seed(args.seed)

        # ---- Resolve experiment directories ----
        self.logs_root = os.path.abspath(args.out_dir)
        self.exp_dir = os.path.join(self.logs_root, args.exp_name)
        self.tb_dir = os.path.join(self.exp_dir, "tensorboard")
        self.ckpt_dir = os.path.join(self.exp_dir, "checkpoints")
        ensure_dir(self.tb_dir)
        ensure_dir(self.ckpt_dir)

        # Save run config snapshot for reproducibility
        with open(os.path.join(self.exp_dir, "run_config.json"), "w") as f:
            json.dump(vars(args), f, indent=2, ensure_ascii=False)

        # TensorBoard writer under experiment dir
        self.writer = SummaryWriter(log_dir=self.tb_dir)

        if torch.cuda.is_available():
            self.device = torch.device("cuda")
            print(f"[INFO] Using GPU: {torch.cuda.get_device_name(0)}")
        else:
            self.device = torch.device("cpu")
            print("[WARNING] CUDA not available. Using CPU instead.")

        # ---- Data & Label map ----
        train_df = load_csv(args.train_csv)
        valid_df = load_csv(args.valid_csv)
        label_map = build_label_map(train_df, valid_df)
        self.label_map = label_map
        self.inv_label_map = {v: k for k, v in label_map.items()}

        # save label map in experiment dir
        with open(os.path.join(self.exp_dir, "label_map.json"), "w") as f:
            json.dump(self.label_map, f, indent=2, ensure_ascii=False)

        self.num_classes = len(self.label_map)
        print(f"[INFO] #speakers (classes) = {self.num_classes}")
        print(f"[INFO] Device: {self.device}")
        print(f"[INFO] Experiment dir: {self.exp_dir}")

        # ---- Datasets ----
        self.train_set = SpeakerDataset(
            csv_path=args.train_csv,
            dataset_root=args.dataset_root,
            label_map=self.label_map,
            target_sr=args.sr,
            duration_sec=args.duration,
            n_mels=args.n_mels,
            n_fft=args.n_fft,
            hop_length=args.hop_length,
            apply_cmvn=True,
        )
        self.valid_set = SpeakerDataset(
            csv_path=args.valid_csv,
            dataset_root=args.dataset_root,
            label_map=self.label_map,
            target_sr=args.sr,
            duration_sec=args.duration,
            n_mels=args.n_mels,
            n_fft=args.n_fft,
            hop_length=args.hop_length,
            apply_cmvn=True,
        )

        # ---- Dataloaders ----
        self.train_loader = DataLoader(
            self.train_set, batch_size=args.batch_size, shuffle=True,
            num_workers=args.num_workers, pin_memory=True, drop_last=True
        )
        self.valid_loader = DataLoader(
            self.valid_set, batch_size=args.batch_size, shuffle=False,
            num_workers=args.num_workers, pin_memory=True, drop_last=False
        )

        # ---- Model/Opt ----
        self.model = MFAConformerSpeakerNet(
            n_classes=self.num_classes,
            input_dim=args.n_mels,
            encoder_dim=args.encoder_dim,
            num_blocks=args.num_blocks,
            num_heads=args.num_heads,
            ff_expansion=args.ff_expansion,
            conv_kernel_size=args.conv_kernel_size,
            dropout=args.dropout
        ).to(self.device)

        # Print model parameter count
        total_params = sum(p.numel() for p in self.model.parameters())
        print(f"[INFO] Total parameters: {total_params:,}")

        # ---- Loss function: Label Smoothing ----
        if args.label_smoothing > 0:
            self.criterion = LabelSmoothingCrossEntropy(smoothing=args.label_smoothing)
            print(f"[INFO] Using Label Smoothing with smoothing={args.label_smoothing}")
        else:
            self.criterion = nn.CrossEntropyLoss()
            print(f"[INFO] Using standard CrossEntropyLoss")

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=4, gamma=0.5)

        self.global_step = 0
        self.best_acc = 0.0

        # CSV metrics path (under experiment dir)
        self.metrics_csv = os.path.join(self.exp_dir, "metrics_log.csv")

        # tqdm common bar format
        self.bar_format = (
            "{l_bar}{bar:30}| {n_fmt}/{total_fmt} "
            "[{elapsed}<{remaining}, {rate_fmt}]"
        )

    def train_one_epoch(self, epoch: int):
        self.model.train()

        total_seen = 0
        total_correct = 0
        loss_sum = 0.0

        lr_now = get_lr(self.optimizer)
        pbar = tqdm(
            self.train_loader,
            desc=f"Train {epoch:03d} | lr={lr_now:.2e}",
            ncols=116,
            bar_format=self.bar_format,
            mininterval=0.5
        )

        tic = time.time()
        for mel, label in pbar:
            bs = label.size(0)
            mel = mel.to(self.device)
            label = label.to(self.device)

            logits = self.model(mel)
            loss = self.criterion(logits, label)

            self.optimizer.zero_grad(set_to_none=True)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.grad_clip)
            self.optimizer.step()

            with torch.no_grad():
                pred = logits.argmax(dim=1)
                correct = (pred == label).sum().item()

                total_seen += bs
                total_correct += correct
                loss_sum += loss.item() * bs

                elapsed = max(1e-9, time.time() - tic)
                ips = total_seen / elapsed
                cur_loss = loss_sum / max(1, total_seen)
                cur_acc = total_correct / max(1, total_seen)

            pbar.set_postfix(
                loss=f"{cur_loss:7.4f}",
                acc=f"{cur_acc:6.4f}",
                ips=f"{ips:6.1f}/s",
                lr=f"{get_lr(self.optimizer):.2e}",
            )
            self.global_step += 1

        epoch_loss = loss_sum / max(1, total_seen)
        epoch_acc = total_correct / max(1, total_seen)
        return epoch_loss, epoch_acc

    @torch.no_grad()
    def evaluate(self, epoch: int):
        self.model.eval()

        total_seen = 0
        total_correct = 0
        loss_sum = 0.0

        pbar = tqdm(
            self.valid_loader,
            desc=f"Valid {epoch:03d}",
            ncols=116,
            bar_format=self.bar_format,
            mininterval=0.5
        )
        tic = time.time()
        for mel, label in pbar:
            bs = label.size(0)
            mel = mel.to(self.device)
            label = label.to(self.device)

            logits = self.model(mel)
            loss = self.criterion(logits, label)

            pred = logits.argmax(dim=1)
            correct = (pred == label).sum().item()

            total_seen += bs
            total_correct += correct
            loss_sum += loss.item() * bs

            elapsed = max(1e-9, time.time() - tic)
            ips = total_seen / elapsed
            cur_loss = loss_sum / max(1, total_seen)
            cur_acc = total_correct / max(1, total_seen)

            pbar.set_postfix(
                val_loss=f"{cur_loss:7.4f}",
                val_acc=f"{cur_acc:6.4f}",
                ips=f"{ips:6.1f}/s",
            )

        epoch_loss = loss_sum / max(1, total_seen)
        epoch_acc = total_correct / max(1, total_seen)
        return epoch_loss, epoch_acc

    def save_checkpoint(self, epoch: int):
        """Save only best.pth (when improved)."""
        state = {
            "epoch": epoch,
            "model_state": self.model.state_dict(),
            "optimizer_state": self.optimizer.state_dict(),
            "scheduler_state": self.scheduler.state_dict(),
            "best_acc": self.best_acc,
            "label_map": self.label_map,
            "args": vars(self.args),
        }
        best_path = os.path.join(self.ckpt_dir, "best.pth")
        torch.save(state, best_path)
        print(f"[Checkpoint] BEST updated → {best_path}")

    def run(self):
        total_epochs = self.args.epochs
        for epoch in range(1, total_epochs + 1):
            print_epoch_header(epoch, total_epochs)
            t0 = time.time()

            train_loss, train_acc = self.train_one_epoch(epoch)
            val_loss, val_acc = self.evaluate(epoch)

            # TensorBoard
            self.writer.add_scalar("train/loss", train_loss, epoch)
            self.writer.add_scalar("train/acc", train_acc, epoch)
            self.writer.add_scalar("valid/loss", val_loss, epoch)
            self.writer.add_scalar("valid/acc", val_acc, epoch)
            self.writer.add_scalar("train/lr", get_lr(self.optimizer), epoch)

            # Scheduler step AFTER logging current LR
            self.scheduler.step()
            cur_lr = get_lr(self.optimizer)

            # Best & CKPT
            is_best = val_acc > self.best_acc
            if is_best:
                self.best_acc = val_acc
                self.save_checkpoint(epoch)

            elapsed = time.time() - t0
            print_epoch_summary(
                epoch=epoch,
                train_loss=train_loss, train_acc=train_acc,
                val_loss=val_loss, val_acc=val_acc,
                best_acc=self.best_acc, lr=cur_lr,
                elapsed=elapsed, ckpt_best=is_best,
                csv_path=self.metrics_csv
            )

        print("[INFO] Training finished.")

---

### 테스터

In [ ]:
def load_label_map(exp_dir: str) -> Dict[str, int]:
    lm_path = os.path.join(exp_dir, "label_map.json")
    if not os.path.exists(lm_path):
        raise FileNotFoundError(f"label_map.json not found at {lm_path}")
    with open(lm_path, "r") as f:
        return json.load(f)


def load_checkpoint(exp_dir: str, device: torch.device):
    ckpt_path = os.path.join(exp_dir, "checkpoints", "best.pth")
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(f"best.pth not found at {ckpt_path}")
    state = torch.load(ckpt_path, map_location=device)
    return state


def infer(args, use_tta=False, tta_crops=5):
    """
    Test 데이터 추론

    Args:
        args: 하이퍼파라미터
        use_tta: TTA 사용 여부
        tta_crops: TTA crop 횟수
    """
    # 경로 설정
    logs_root = os.path.abspath(args.out_dir)
    exp_dir = os.path.join(logs_root, args.exp_name)
    ensure_dir(exp_dir)

    # Label map 로드
    label_map = load_label_map(exp_dir)
    inv_label_map = {v: k for k, v in label_map.items()}
    num_classes = len(inv_label_map)

    # Device 설정
    device = torch.device("cuda" if torch.cuda.is_available() and not args.cpu else "cpu")
    print(f"[INFO] Device: {device}")
    print(f"[INFO] Experiment dir: {exp_dir}")
    if use_tta:
        print(f"[INFO] Using TTA with {tta_crops} crops")

    # 데이터셋 준비
    dataset = TestDataset(
        csv_path=args.test_csv,
        dataset_root=args.dataset_root,
        target_sr=args.sr,
        duration_sec=args.duration,
        n_mels=args.n_mels,
        n_fft=args.n_fft,
        hop_length=args.hop_length,
        apply_cmvn=True,
    )
    loader = DataLoader(
        dataset, batch_size=args.batch_size, shuffle=False,
        num_workers=args.num_workers, pin_memory=True, drop_last=False,
    )

    # 모델 로드
    model = MFAConformerSpeakerNet(
        n_classes=num_classes,
        input_dim=args.n_mels,
        encoder_dim=args.encoder_dim,
        num_blocks=args.num_blocks,
        num_heads=args.num_heads,
        ff_expansion=args.ff_expansion,
        conv_kernel_size=args.conv_kernel_size,
        dropout=args.dropout
    ).to(device)
    state = load_checkpoint(exp_dir, device)
    model.load_state_dict(state["model_state"], strict=True)
    model.eval()

    # 추론
    filenames = []
    preds = []

    with torch.no_grad():
        pbar = tqdm(loader, desc="Inference", ncols=116)
        for mel, filename in pbar:
            mel = mel.to(device)

            if use_tta:
                # TTA: 여러 번 crop해서 평균
                B, C, F, T = mel.shape
                target_len = int(args.duration * args.sr / args.hop_length) + 1

                logits_list = []
                for _ in range(tta_crops):
                    if T > target_len:
                        start = random.randint(0, T - target_len)
                        mel_crop = mel[:, :, :, start:start + target_len]
                    else:
                        mel_crop = mel

                    logits = model(mel_crop)
                    logits_list.append(logits)

                logits = torch.stack(logits_list).mean(dim=0)
            else:
                logits = model(mel)

            pred_idx = logits.argmax(dim=1).cpu().numpy().tolist()
            pred_spk = [inv_label_map[int(i)] for i in pred_idx]

            filenames.extend(filename)
            preds.extend(pred_spk)

    # CSV 저장 (logs/predictions/ 폴더에)
    predictions_dir = os.path.join(logs_root, "predictions")
    ensure_dir(predictions_dir)

    out_csv = os.path.join(predictions_dir, f"test_predictions_{args.exp_name}.csv")
    df = pd.DataFrame({"path": filenames, "speaker_id": preds})
    df.to_csv(out_csv, index=False)
    print(f"[INFO] Saved predictions → {out_csv}")
    print("\n저장된 파일을 런어스 자료실에 제출해주세요!")


def ensemble_infer(exp_names, use_tta=True, tta_crops=10):
    """
    여러 모델의 예측을 평균내는 Ensemble Inference

    Args:
        exp_names: 실험 이름 리스트
        use_tta: TTA 사용 여부
        tta_crops: TTA crop 횟수
    """
    args = build_args()

    logs_root = os.path.abspath(args[0].out_dir)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Label map 로드
    exp_dir = os.path.join(logs_root, exp_names[0])
    label_map = load_label_map(exp_dir)
    inv_label_map = {v: k for k, v in label_map.items()}
    num_classes = len(inv_label_map)

    print(f"[INFO] Loading {len(exp_names)} models for ensemble...")

    # 모든 모델 로드
    models = []
    for exp_name in exp_names:
        exp_dir = os.path.join(logs_root, exp_name)

        model = MFAConformerSpeakerNet(
            n_classes=num_classes,
            input_dim=args[0].n_mels,
            encoder_dim=args[0].encoder_dim,
            num_blocks=args[0].num_blocks,
            num_heads=args[0].num_heads,
            ff_expansion=args[0].ff_expansion,
            conv_kernel_size=args[0].conv_kernel_size,
            dropout=args[0].dropout
        ).to(device)

        state = load_checkpoint(exp_dir, device)
        model.load_state_dict(state["model_state"], strict=True)
        model.eval()
        models.append(model)
        print(f"  ✓ Loaded: {exp_name}")

    print(f"[INFO] Ensemble with {len(models)} models + TTA {tta_crops}x")

    # 데이터셋 준비
    dataset = TestDataset(
        csv_path=args[0].test_csv,
        dataset_root=args[0].dataset_root,
        target_sr=args[0].sr,
        duration_sec=args[0].duration,
        n_mels=args[0].n_mels,
        n_fft=args[0].n_fft,
        hop_length=args[0].hop_length,
        apply_cmvn=True,
    )
    loader = DataLoader(
        dataset, batch_size=args[0].batch_size, shuffle=False,
        num_workers=args[0].num_workers, pin_memory=True, drop_last=False,
    )

    # 추론
    filenames = []
    preds = []

    with torch.no_grad():
        pbar = tqdm(loader, desc="Ensemble Inference", ncols=116)
        for mel, filename in pbar:
            mel = mel.to(device)

            # 모든 모델의 예측 수집
            all_logits = []

            for model in models:
                if use_tta:
                    # TTA
                    B, C, F, T = mel.shape
                    target_len = int(args[0].duration * args[0].sr / args[0].hop_length) + 1

                    logits_list = []
                    for _ in range(tta_crops):
                        if T > target_len:
                            start = random.randint(0, T - target_len)
                            mel_crop = mel[:, :, :, start:start + target_len]
                        else:
                            mel_crop = mel

                        logits = model(mel_crop)
                        logits_list.append(logits)

                    logits = torch.stack(logits_list).mean(dim=0)
                else:
                    logits = model(mel)

                all_logits.append(logits)

            # 모든 모델의 logits 평균
            ensemble_logits = torch.stack(all_logits).mean(dim=0)
            pred_idx = ensemble_logits.argmax(dim=1).cpu().numpy().tolist()
            pred_spk = [inv_label_map[int(i)] for i in pred_idx]

            filenames.extend(filename)
            preds.extend(pred_spk)

    # CSV 저장 (logs/predictions/ 폴더에 - Ensemble은 고정 이름)
    predictions_dir = os.path.join(logs_root, "predictions")
    ensure_dir(predictions_dir)

    out_csv = os.path.join(predictions_dir, "test_predictions.csv")
    df = pd.DataFrame({"path": filenames, "speaker_id": preds})
    df.to_csv(out_csv, index=False)
    print(f"[INFO] Saved ensemble predictions → {out_csv}")
    print("\n저장된 파일을 런어스 자료실에 제출해주세요!")

---

### 모델 학습 실행하기

In [ ]:
!mkdir -p /content/drive/MyDrive/dsp_dataset/dataset
!cp -r /content/drive/MyDrive/dsp_dataset/dataset/logs /content/logs

# Model 1: Seed 42 (강한 Regularization)
args = build_args()
args[0].exp_name = "EXP_StrongReg_seed42"
args[0].label_smoothing = 0.15  # 0.1 → 0.15
args[0].dropout = 0.2           # 0.1 → 0.2
args[0].weight_decay = 2e-4     # 1e-4 → 2e-4
args[0].seed = 42
args[0].epochs = 25

print("=" * 66)
print("[Model 1/3] Training with seed=42")
print("=" * 66)
trainer = Trainer(args[0])
trainer.run()

# Model 2: Seed 123
args = build_args()
args[0].exp_name = "EXP_StrongReg_seed123"
args[0].label_smoothing = 0.15
args[0].dropout = 0.2
args[0].weight_decay = 2e-4
args[0].seed = 123
args[0].epochs = 25

print("=" * 66)
print("[Model 2/3] Training with seed=123")
print("=" * 66)
trainer = Trainer(args[0])
trainer.run()

# Model 3: Seed 777
args = build_args()
args[0].exp_name = "EXP_StrongReg_seed777"
args[0].label_smoothing = 0.15
args[0].dropout = 0.2
args[0].weight_decay = 2e-4
args[0].seed = 777
args[0].epochs = 25

print("=" * 66)
print("[Model 3/3] Training with seed=777")
print("=" * 66)
trainer = Trainer(args[0])
trainer.run()

print("\n" + "=" * 66)
print("✓ All 3 models trained! Ready for ensemble.")
print("=" * 66)

---

### 모델 테스트 실행하기

In [ ]:
ensemble_infer(
    exp_names=[
        "EXP_StrongReg_seed42",
        "EXP_StrongReg_seed123",
        "EXP_StrongReg_seed777"
    ],
    use_tta=True,
    tta_crops=10
)
# 결과 파일: logs/predictions/test_predictions.csv

---

#### [결과 분석] 텐서보드 활용하기

In [ ]:
# Initialize directories for logging and model storage
DIR_NAME = os.path.join(os.getcwd(), 'logs', f'{args[0].exp_name}')

%load_ext tensorboard
%tensorboard --logdir {DIR_NAME}

데이터 확인

In [ ]:
import librosa
import matplotlib.pyplot as plt
import pandas as pd

# ===== 1. CSV 로드 =====
train_df = pd.read_csv('./dataset/train.csv')
valid_df = pd.read_csv('./dataset/valid.csv')
test_df = pd.read_csv('./dataset/test.csv')

print("="*60)
print("데이터셋 정보")
print("="*60)
print(f"Train samples: {len(train_df)}")
print(f"Valid samples: {len(valid_df)}")
print(f"Test samples: {len(test_df)}")
print(f"\nUnique speakers in train: {train_df['speaker_id'].nunique()}")
print("\nTrain 샘플 예시:")
print(train_df.head())

# ===== 2. 데이터 확인 =====
print("\n" + "="*60)
print("오디오 데이터 확인")
print("="*60)

sample_path = train_df.iloc[0]['path']
full_path = os.path.join('./dataset', sample_path)
y, sr = librosa.load(full_path, sr=16000)
mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=80, n_fft=400, hop_length=160)
mel_db = librosa.power_to_db(mel)

print(f"Sample path: {sample_path}")
print(f"Speaker: {train_df.iloc[0]['speaker_id']}")
print(f"Audio length: {len(y)/sr:.2f}s ({len(y)} samples)")
print(f"Mel shape: {mel.shape} (n_mels, time_frames)")
print(f"Mel dB range: {mel_db.min():.2f} ~ {mel_db.max():.2f}")

plt.figure(figsize=(12, 4))
plt.imshow(mel_db, aspect='auto', origin='lower', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title(f"Mel-spectrogram: {train_df.iloc[0]['speaker_id']}")
plt.xlabel('Time frames')
plt.ylabel('Mel frequency bins')
plt.tight_layout()
plt.show()

# ===== 3. 모델 출력 확인 =====
print("\n" + "="*60)
print("모델 출력 확인")
print("="*60)

# Trainer에서 만든 데이터로더 사용
if 'trainer' in globals():
    loader = trainer.train_loader
    model = trainer.model
    device = trainer.device

    for mel_batch, label_batch in loader:
        print(f"Batch mel shape: {mel_batch.shape}  # [batch, 1, n_mels, time]")
        print(f"Batch label shape: {label_batch.shape}  # [batch]")
        print(f"Mel range: {mel_batch.min():.4f} ~ {mel_batch.max():.4f}")
        print(f"Labels in batch: {label_batch[:10].tolist()}")

        # 모델 순전파
        mel_batch = mel_batch.to(device)
        model.eval()
        with torch.no_grad():
            logits = model(mel_batch)

        print(f"\nModel output (logits) shape: {logits.shape}  # [batch, n_classes]")
        print(f"Logits range: {logits.min():.4f} ~ {logits.max():.4f}")
        print(f"Predictions: {logits.argmax(dim=1)[:10].tolist()}")
        print(f"Ground truth: {label_batch[:10].tolist()}")

        # 정확도 확인
        preds = logits.argmax(dim=1).cpu()
        correct = (preds == label_batch).sum().item()
        acc = correct / len(label_batch)
        print(f"\nBatch accuracy: {acc*100:.2f}%")

        break
else:
    print("[WARN] Trainer가 아직 생성되지 않았습니다. 먼저 학습을 실행하세요.")